##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# HParams 대시보드를 이용한 하이퍼 매개변수 튜닝

<table class="tfo-notebook-buttons" align="left">
  <td>
<img src="https://www.tensorflow.org/images/tf_logo_32px.png"><a target="_blank" href="https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams">TensorFlow.org에서 보기</a>
</td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>
  </td>
  <td>
<img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"><a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/hyperparameter_tuning_with_hparams.ipynb">GitHub에서 소스 보기</a>
</td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

머신러닝 모델을 빌드할 때 레이어의 드롭아웃 비율 또는 학습률과 같은 다양한 [하이퍼 매개변수](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning))를 선택해야 합니다. 이러한 결정은 정확성과 같은 모델 메트릭에 영향을 미칩니다. 따라서 머신러닝 워크플로에서 중요한 단계는 문제에 가장 적합한 하이퍼 매개변수를 식별하는 것이며, 이를 위해 종종 실험이 필요합니다. 이 프로세스를 "하이퍼 매개변수 최적화" 또는 "하이퍼 매개변수 튜닝"이라고 합니다.

TensorBoard의 HParams 대시보드에는 최상의 실험 또는 가장 유망한 하이퍼 매개변수 세트를 찾는 과정에 유용하게 이용할 수 있는 다양한 도구들이 있습니다.

이 튜토리얼에서는 다음 단계에 중점을 둡니다.

1. 설정 및 HParams 요약을 실험합니다.
2. TensorFlow 실행을 조정하여 하이퍼 매개변수와 메트릭을 로깅합니다.
3. 실행을 시작하고 하나의 상위 디렉토리에 모두 로깅합니다.
4. TensorBoard의 HParams 대시보드에서 결과를 시각화합니다.

참고: HParams 요약 API와 대시보드 UI는 미리 보기 단계이며 앞으로 변경이 이루어질 것입니다.

우선, TF 2.0을 설치하고 TensorBoard 노트북 확장을 로드합니다.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Import TensorFlow and the TensorBoard HParams plugin:

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

[FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) 데이터세트를 다운로드하고 범위를 조정합니다.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


## 1. 설정 및 HParams 실험 요약 실험하기

모델에서 세 가지 하이퍼 매개변수로 실험합니다.

1. 첫 번째 밀집 레이어의 단위 수
2. 드롭아웃 레이어의 드롭아웃 비율
3. 옵티마이저

시도할 값을 나열하고 TensorBoard에 실험 구성을 로깅합니다. 선택적 단계로, 도메인 정보를 제공하여 UI에서 하이퍼 매개변수를 보다 정밀하게 필터링하고 표시할 메트릭을 지정할 수 있습니다.

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

이 단계를 건너뛰려면 `HParam` 값이 사용되었을 위치에 문자열 리터럴을 사용할 수 있습니다(예: `hparams[HP_DROPOUT]` 대신 `hparams['dropout']`).

## 2. TensorFlow 실행을 조정하여 하이퍼 매개변수 및 메트릭 로깅하기

모델은 사이에 드롭아웃 레이어가 있는 두 개의 밀집 레이어로 매우 간단합니다. 하이퍼 매개변수가 더 이상 하드 코딩되지 않지만 훈련 코드는 익숙해 보일 것입니다. 대신, 하이퍼 매개변수는 `hparams` 사전에 제공되며 훈련 함수 전체에서 사용됩니다.

In [ ]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

각 실행에 대해, 하이퍼 매개변수 및 최종 정확성과 함께 hparams 요약을 로깅합니다.

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

Keras 모델을 훈련할 때 직접 작성하는 대신 콜백을 사용할 수 있습니다.

```python
model.fit(
    ...,
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ],
)
```

## 3. 실행을 시작하고 하나의 상위 디렉토리에 모두 로깅하기

이제 여러 실험을 시도하여 각 실험에서 서로 다른 하이퍼 매개변수 세트로 훈련할 수 있습니다.

간단하게 하기 위해 그리드 검색을 사용합니다(이산 매개변수의 모든 조합과 실수 값 매개변수의 하한 및 상한만 시도). 더 복잡한 시나리오의 경우, 각 하이퍼 매개변수 값을 임의로 선택하는 것이 더 효과적일 수 있습니다(무작위 검색이라고 함). 더 발전된 방법들을 이용할 수 있습니다.

몇 가지 실험을 실행합니다. 몇 분 정도 소요됩니다.

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 35us/sample - loss: 0.4806 - accuracy: 0.8321
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.6519 - accuracy: 0.7770
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.5309 - accuracy: 0.8154
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.7007 - accuracy: 0.7683
--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 38us/sample - loss: 0.4645 - accuracy: 0.8395
--- Starting trial: run-5
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'sgd'}
10000/10000 [========================

## 4. TensorBoard의 HParams 플러그인에서 결과 시각화하기

이제 HParams 대시보드를 열 수 있습니다. TensorBoard를 시작하고 상단의 "HParams"를 클릭합니다.

In [ ]:
%tensorboard --logdir logs/hparam_tuning

<!-- <img class="tfo-display-only-on-site" src="images/hparams_table.png?raw=1"/> -->

대시보드의 왼쪽 창에는 HParams 대시보드의 모든 뷰에서 활성화되는 필터링 기능이 있습니다.

- 대시보드에 표시할 하이퍼 매개변수/메트릭 필터링
- 대시보드에 표시할 하이퍼 매개변수/메트릭 값 필터링
- 실행 상태 필터링(실행 중, 성공, ...)
- 테이블 뷰에서 하이퍼 매개변수/메트릭별로 정렬
- 표시할 세션 그룹의 수(실험이 많은 경우 성능 관리에 유용)


HParams 대시보드에는 다양하고 유용한 정보가 포함된 세 가지 뷰가 있습니다

- **테이블 뷰**에는 실행과 해당 실행의 하이퍼 매개변수 및 메트릭이 나열됩니다.
- **평행 좌표 보기**에는 각 실행이 각 하이퍼 매개변수와 메트릭에 대한 축을 통과하는 선으로 표시됩니다. 축을 마우스로 클릭하고 끌면 이 축을 통과하는 실행만 강조 표시하는 영역이 표시됩니다. 그러면 가장 중요한 하이퍼 매개변수 그룹을 보다 쉽게 식별할 수 있습니다. 축 자체는 끌어서 재정렬할 수 있습니다.
- **산포도 보기**는 각 하이퍼 매개변수/메트릭과 각 메트릭을 비교하는 도표를 보여줍니다. 상관 관계를 이해하는 데 도움이 될 수 있습니다. 클릭하고 끌어서 특정 플롯의 영역을 선택하고 다른 플롯에서 해당 세션을 강조 표시합니다.

테이블 행, 평행 좌표선 및 산포도 마켓을 클릭하면 해당 세션에 대한 훈련 스텝의 함수로 메트릭을 나타낸 플롯을 볼 수 있습니다. 하지만 이 튜토리얼에서는 각 실행에 대해 하나의 스텝만 사용합니다.

HParams 대시보드의 기능을 더 자세히 살펴보려면 더 많은 실험이 포함된 사전 생성된 로그 세트를 다운로드하세요.

In [ ]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

View these logs in TensorBoard:

In [ ]:
%tensorboard --logdir logs/hparam_demo

<!-- <img class="tfo-display-only-on-site" src="images/hparams_parallel_coordinates.png?raw=1"/> -->

HParams 대시보드에서 다양한 뷰를 사용해 볼 수 있습니다.

예를 들어, 'Parallel Coorndiates View'로 이동하고 정확성 축을 클릭하고 끌어서 정확성이 가장 높은 실행을 선택할 수 있습니다. 이러한 실행이 옵티마이저 축에서 'adam'을 통과하므로 이들 실험에서 'adam'이 'sgd'보다 성능이 더 우수하다는 결론을 내릴 수 있습니다.